# I have added the comments for better understanding and tried to write the code in similar way as we are taught in theory in class first cell is 2 A answer and second cell is 2 B answer ( i have used different methods for each and called with different variables in both as i was having certain problems and to solve them i did so ) , third cell is 2 C answer now give this in python markdown code

In [10]:
#2 A )


# Aswwer



import numpy as np

# Define grid world dimensions and cell types
rows, cols = 4, 5
alpha = 0.99  # Discount factor
convergence_threshold = 1e-6  # Convergence threshold

# Define costs based on cell colors
costs = {
    "violet": -1,
    "white": 0,
    "green": 1,
    "red": 10,
    "gray": None  # gray cells are obstacles
}

# Create the grid with costs based on the given figure
grid = [
    ["white", "white", "green", "white", "violet"],
    ["white", "gray", "green", "white", "red"],
    ["white", "red", "gray", "green", "white"],
    ["green", "white", "white", "red", "violet"],
]

# Create cost matrix
cost_matrix = np.zeros((rows, cols))
for i in range(rows):
    for j in range(cols):
        if grid[i][j] != "gray":
            cost_matrix[i, j] = costs[grid[i][j]]
        else:
            cost_matrix[i, j] = None  # Set obstacles as None

# Define possible actions (up, down, left, right, stay)
actions = {
    "up": (-1, 0),
    "down": (1, 0),
    "left": (0, -1),
 "right": (0, 1),
    "stay": (0, 0)
}

# Initialize value function for each cell to zero
value_function = np.zeros((rows, cols))

def is_valid_position(row, col):
    """Check if a position is within bounds and not a gray cell."""
    return 0 <= row < rows and 0 <= col < cols and grid[row][col] != "gray"

def value_iteration():
    """Perform value iteration to find the optimal value function and policy."""
    global value_function
    iteration = 0

    # To store the best action at each position
    policy = np.empty((rows, cols), dtype=object)

    while True:
        delta = 0
        new_value_function = value_function.copy()

        for i in range(rows):
            for j in range(cols):
                if grid[i][j] == "gray":
                    continue
                
                # Compute value for each action
                action_values = []
                for action, (di, dj) in actions.items():
                    ni, nj = i + di, j + dj
                    if is_valid_position(ni, nj):
                        action_value = cost_matrix[i, j] + alpha * value_function[ni, nj]
                    else:
                        action_value = cost_matrix[i, j] + alpha * value_function[i, j]  # Stay in place if action is invalid
                    action_values.append((action_value, action))

                # Update value function for cell (i, j)
                new_value_function[i, j] = min(action_values, key=lambda x: x[0])[0]

                # Store the best action
                best_action_value, best_action = min(action_values, key=lambda x: x[0])
                
                # If value is less than -99.999 ( that is 100 ), set the action to 'stay'
                if best_action_value < -99.999:
                    best_action = "stay"

                # Assign the best action based on the value
                policy[i, j] = best_action

                delta = max(delta, abs(new_value_function[i, j] - value_function[i, j]))

        value_function = new_value_function
        iteration += 1

        if delta < convergence_threshold:
            break

    return iteration, policy

# Perform Value Iteration
vi_iterations, final_policy = value_iteration()

# Adjust the final value function according to the specified changes
final_value_function = np.where(value_function < -99.99999, 'stay', value_function)
final_value_function[final_value_function == 0] = None

# Print final results
print("Value Iteration Converged in:", vi_iterations, "iterations")
print("\nFinal Value Function after Value Iteration (Cost-to-Go for Each Cell):")
print(final_value_function)

print("\nFinal Policy (Best Action at Each Cell):")
print(final_policy)


Value Iteration Converged in: 1376 iterations

Final Value Function after Value Iteration (Cost-to-Go for Each Cell):
[['-95.07940236574548' '-96.03980136574546' '-97.00990136574546'
  '-98.99990136574546' '-99.99990136574546']
 ['-94.12860735574546' '0.0' '-96.02980136574546' '-98.00990136574546'
  '-88.99990136574546']
 ['-93.18732029584545' '-82.25544610654447' '0.0' '-97.00990136574546'
  '-98.99990136574546']
 ['-91.25544610654447' '-90.34289065913646' '-89.43946076620256'
  '-88.99990136574546' '-99.99990136574546']]

Final Policy (Best Action at Each Cell):
[['right' 'right' 'right' 'right' 'stay']
 ['up' None 'right' 'up' 'up']
 ['up' 'left' None 'right' 'down']
 ['up' 'left' 'left' 'right' 'stay']]


In [9]:
#2 B)

###ANSWER

import numpy as np

# Define constants and parameters
ROWS, COLS = 4, 5
ALPHA = 0.99  # Discount factor
CONVERGENCE_THRESHOLD = 1e-6  # Convergence threshold

# Define costs based on cell types
costs = {
    "violet": -1,
    "white": 0,
    "green": 1,
    "red": 10,
    "gray": None  # gray cells represent obstacles
}

# Define the grid layout
grid = [
    ["white", "white", "green", "white", "violet"],
    ["white", "gray", "green", "white", "red"],
    ["white", "red", "gray", "green", "white"],
    ["green", "white", "white", "red", "violet"],
]

# Create cost matrix
def create_cost_matrix(grid):
    """Convert grid with cell colors to a numerical cost matrix."""
    cost_matrix = np.full((ROWS, COLS), np.nan)  # Initialize with np.nan for obstacles
    for i in range(ROWS):
        for j in range(COLS):
            if grid[i][j] != "gray":
                cost_matrix[i, j] = costs[grid[i][j]]
    return cost_matrix

# Action definitions
actions = {
    "up": (-1, 0),
    "down": (1, 0),
    "left": (0, -1),
    "right": (0, 1),
    "stay": (0, 0),
}

def is_valid_position(row, col):
    """Check if the position is within bounds and not an obstacle."""
    return 0 <= row < ROWS and 0 <= col < COLS and grid[row][col] != "gray"

def initialize_A(cost_matrix):
    """Initialize the A matrix based on the valid positions."""
    num_states = np.sum(~np.isnan(cost_matrix))  # Count valid states
    A = np.zeros((num_states, num_states))
    return A

def update_A(A, value_function, index):
    """Update the A matrix row for the current state."""
    j, k = index
    J_nothing = value_function[j, k]
    J_values = {"stay": J_nothing}

    # Calculate values for each action
    for action, (di, dj) in actions.items():
        ni, nj = j + di, k + dj
        if is_valid_position(ni, nj):
            J_values[action] = value_function[ni, nj]
        else:
            J_values[action] = J_nothing  # If action is invalid, stay

    # Find best action
    min_cost_action = min(J_values, key=J_values.get)
    action_representation = np.zeros((ROWS, COLS))

    if min_cost_action == "stay":
        action_representation[j, k] = 1
    else:
        ni, nj = (j + actions[min_cost_action][0]), (k + actions[min_cost_action][1])
        action_representation[ni, nj] = 1

    non_nan_idx = np.flatnonzero(~np.isnan(cost_matrix))  # Get non-NaN indices
    A[np.where(non_nan_idx == np.ravel_multi_index((j, k), cost_matrix.shape))[0][0]] = action_representation.flatten()[non_nan_idx]

def grid_world_policy_iter(max_iter, tol, alpha, g):
    """Perform policy iteration on the grid-world to compute the optimal value function."""
    iterations = np.zeros(max_iter + 1)  # Renamed this variable
    mask = ~np.isnan(g)
    
    # Initialize A
    A = initialize_A(g)
    I = np.eye(np.sum(mask))

    for i in range(max_iter):
        # Solve for J_mu using current policy
        J_mu = np.linalg.solve(I - alpha * A, g[mask])
        
        # Reshape J_mu to original grid
        J_full = np.full(g.shape, np.nan)
        J_full[mask] = J_mu
        
        # Compute the norm of the value function
        iterations[i] = np.nanmean(J_full**2)  # Use mean to avoid NaN issues
        
        # Ensure the optimal policy has only positive values
        if np.all(J_full >= 0):
            break
        
        # Check for convergence
        if i > 0 and abs(iterations[i] - iterations[i - 1]) < tol:
            break
        
        # Reset A for the next iteration
        A = initialize_A(g)
        
        # Update matrix A with new state evaluations
        for j, k in np.ndindex(g.shape):
            if not np.isnan(J_full[j, k]):
                update_A(A, J_full, (j, k))

    # Truncate the iterations history
    iterations = iterations[:i + 1]
    
    return J_full, iterations

# Running the policy iteration
cost_matrix = create_cost_matrix(grid)
optimal_values, convergence_history = grid_world_policy_iter(max_iter=1000, tol=1e-5, alpha=ALPHA, g=cost_matrix)

# Print outputs
print("\nOptimal Value Function:")
print(optimal_values)

# Number of iterations
print("\nNumber of iterations:")
print(len(convergence_history))

# Generate action matrix based on optimal values
def generate_action_matrix(optimal_values):
    action_matrix = np.full((ROWS, COLS), None)  # Fill with None initially
    
    for j, k in np.ndindex(optimal_values.shape):
        if not np.isnan(optimal_values[j, k]):
            # Set action to 'stay' if value is near to -100
            if optimal_values[j, k] <= -99.999:
                action_matrix[j, k] = 'stay'
                continue
            
            J_values = {}
            for action, (di, dj) in actions.items():
                ni, nj = j + di, k + dj
                if is_valid_position(ni, nj):
                    J_values[action] = optimal_values[ni, nj]
                else:
                    J_values[action] = optimal_values[j, k]  # Stay if action is invalid

            best_action = min(J_values, key=J_values.get)
            action_matrix[j, k] = best_action  # Store the best action
    
    return action_matrix

# Generate and print the action matrix
action_matrix = generate_action_matrix(optimal_values)

print("\nAction Matrix (Optimal Actions):")
print(action_matrix)



Optimal Value Function:
[[ -95.08   -96.04   -97.01   -99.    -100.   ]
 [ -94.129      nan  -96.03   -98.01   -89.   ]
 [ -93.187  -82.256      nan  -97.01   -99.   ]
 [ -91.256  -90.343  -89.44   -89.    -100.   ]]

Number of iterations:
11

Action Matrix (Optimal Actions):
[['right' 'right' 'right' 'right' 'stay']
 ['up' None 'right' 'up' 'up']
 ['up' 'left' None 'right' 'down']
 ['up' 'left' 'left' 'right' 'stay']]


# 2 C) ANSWER

# Comparison of Value Iteration and Policy Iteration

## 1. Solution Comparison

### Value Iteration
- **Goal**: Computes the optimal value function directly through iterative updates.
- **Final Output**: Produces an optimal value function representing the expected cost-to-go for each state along with a policy (the best action for each state).
- **Best Action**: The action corresponding to the minimum expected cost at each state after computing the value function.

### Policy Iteration
- **Goal**: Alternates between policy evaluation (calculating the value function based on the current policy) and policy improvement (updating the policy based on the newly found value function).
- **Final Output**: Also yields an optimal value function and an optimal policy, but it often requires fewer iterations to converge compared to value iteration.
- **Action Matrix**: The actions represent the best movement for each state, derived from the value function across iterations.

## 2. Convergence Characteristics

### Value Iteration
- **Convergence**: 
  - The method converges when the maximum change in values across all states is less than a specified threshold (in this case, `1e-6`).
  - Typically, value iteration requires a significant number of iterations, as it recalculates values for all states based on the current values for neighboring states at each iteration.
- **Performance Considerations**:
  - If the discount factor is close to \(1\) (here \(0.99\)), convergence may take longer, as values change slowly.
  
### Policy Iteration
- **Convergence**: 
  - Often converges faster than value iteration because it improves the policy iteratively while evaluating it in the same iteration.
  - Stops when the policy does not change during an iteration, indicating the optimal policy has been found.
- **Performance Considerations**:
  - Fewer iterations are generally needed, making policy iteration potentially more efficient for large state spaces, especially when the policies stabilize quickly.

## 3. Computational Complexity

### Value Iteration
- **Time Complexity**: \(O(n^2 \cdot k)\), where \(n\) is the number of states, and \(k\) is the number of iterations until convergence.
- **Space Complexity**: \(O(n)\), as the value function and policy are stored in arrays proportional to the number of states.

### Policy Iteration
- **Time Complexity**: Although each policy evaluation can also be \(O(n^2)\) (solving a linear system), the overall complexity can be substantially lower in practice due to fewer iterations. If there are \(m\) iterations until convergence for the policy improvement, it may be roughly \(O(m \cdot n^3)\) due to the linear system solver.
- **Space Complexity**: Also \(O(n)\) for storing the value function and policy.

